# SmartHome AI Training


 We set out to train an machine learning system using data from a house in *Sceaux, France*. 47 months of data capture across the house with specific data on 3 rooms was a large training set for us to use.

In [ ]:
!pip install keras

For this machine learning project we used keras with all the basic Python data science packages!

In [2]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

Using TensorFlow backend.


To create the dataset needed to train the data, the data was prepared using the following function. It creates arrays of data for the same variable but shifted by a timestep. When processed in the machine learning algrorithm this forces the network to learn how to predict the next timestep from the previous step's data.

In [3]:

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

To load the dataset we used Pandas to create a dataframe. Pandas can load .csv files directly. A few missing data points were contained in the .csv files marked by a '?'. These were removed using `dataset = dataset.replace({'?':0.0} )`. All data was converted to the same datatype to avoid issues later.

In [4]:
# load dataset
dataset = read_csv('new_house.csv', header=0, index_col=0, low_memory=False)
dataset = dataset.replace({'?':0.0} )
values = dataset.values


# ensure all data is float
values = values.astype('float32')



/Users/James/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


      var1(t-1)  var2(t-1)  var3(t-1)   var2(t)   var3(t)   var4(t)   var5(t)  \
1  0.000000e+00   0.379069   0.300719  0.481928  0.313669  0.919260  0.475207   
2  9.536743e-07   0.481928   0.313669  0.483186  0.358273  0.917922  0.475207   
3  9.536743e-07   0.483186   0.358273  0.484445  0.361151  0.919693  0.475207   
4  1.907349e-06   0.484445   0.361151  0.329617  0.379856  0.927326  0.326446   
5  1.907349e-06   0.329617   0.379856  0.316490  0.375540  0.924730  0.309917   

   var6(t)  var7(t)   var8(t)  
1      0.0   0.0125  0.516129  
2      0.0   0.0250  0.548387  
3      0.0   0.0125  0.548387  
4      0.0   0.0125  0.548387  
5      0.0   0.0250  0.548387  


We used some of the *Scikit* tools to further prepare the data. A `MinMaxScaler` for all of the values so that they had a conversion to lie between 0 and 1. The Scaler also creates an inverse version of itself at the same time so that we can get back to it at the end. We decided to save time on training and only predict overall energy usage. 

This was not a simple function of the 3 rooms as there was a lot of hidden data that was not recorded in the study.

In [ ]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
reframed.drop(reframed.columns[[3,4,5,6,7,8]], axis=1, inplace=True)
# drop columns we don't want to predict
print(reframed.head())

## Seperating Training and Testing
---
The overall dataset is huge so we trained using the first $\sim 16$ months of data. A test set was then generated for just a few months following this sample. The training data has to be reshaped so that it can be fed into the *Long Short-Term Memory Cells* or **LSTMs** that are used in the network. 

In [5]:
values = reframed.values
n_train_hours = int(0.3*len(values))
train = values[:n_train_hours, :]
test = values[n_train_hours:int(1.2*n_train_hours), :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(614762, 1, 9) (614762,) (122952, 1, 9) (122952,)


## Training Model
---


Here we define the model that is going to be trained. This is a very simple model comprised of **50 LSTM** cells that feed into a single dense layer. This dense layer outputs a single value; the value we want for the next time step. The loss function used is the *Mean Absolute Error*.

$mae = |PredictedValue-TrueValue|$

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')


## Callback Systems
---


In [7]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

/Users/James/opt/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [ ]:
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, 
                    validation_data=(test_X, test_y), verbose=2, shuffle=False,
                   callbacks=[earlyStopping, mcp_save, reduce_lr_loss])


Train on 614762 samples, validate on 122952 samples
Epoch 1/50
 - 67s - loss: 0.1254 - val_loss: 0.1146
Epoch 2/50
 - 64s - loss: 0.1089 - val_loss: 0.1022
Epoch 3/50
 - 63s - loss: 0.1046 - val_loss: 0.0998
Epoch 4/50
 - 65s - loss: 0.1027 - val_loss: 0.0999
Epoch 5/50
 - 80s - loss: 0.1013 - val_loss: 0.0996
Epoch 6/50
 - 97s - loss: 0.1002 - val_loss: 0.0993
Epoch 7/50
 - 65s - loss: 0.0992 - val_loss: 0.0989
Epoch 8/50
 - 76s - loss: 0.0983 - val_loss: 0.0987
Epoch 9/50
 - 77s - loss: 0.0974 - val_loss: 0.0980
Epoch 10/50
 - 74s - loss: 0.0961 - val_loss: 0.0969
Epoch 11/50
 - 68s - loss: 0.0943 - val_loss: 0.0947
Epoch 12/50
 - 75s - loss: 0.0911 - val_loss: 0.0903
Epoch 13/50
 - 77s - loss: 0.0856 - val_loss: 0.0839
Epoch 14/50
 - 89s - loss: 0.0795 - val_loss: 0.0815
Epoch 15/50
 - 72s - loss: 0.0768 - val_loss: 0.0797
Epoch 16/50
 - 75s - loss: 0.0756 - val_loss: 0.0753
Epoch 17/50
 - 84s - loss: 0.0746 - val_loss: 0.0741
Epoch 18/50
 - 70s - loss: 0.0737 - val_loss: 0.0732
Epo

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
yhat = model.predict(test_X)
print(yhat.shape)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
pyplot.plot(test_X[:,0], inv_y)
pyplot.plot(test_X[:,0], inv_yhat)
pyplot.show()